In [ ]:
pip install anchor-exp

In [2]:
pip install torch transformers spacy && python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.8 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 6.6 MB 60.7 MB/s 
     |████████████████████████████████| 101 kB 14.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import numpy as np
import sklearn
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
import time

In [4]:
import transformers
from transformers import AutoModelForSequenceClassification
# load a BERT sentiment analysis model
tokenizer = transformers.AutoTokenizer.from_pretrained("ProsusAI/finbert")
# model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").cuda()
cl_path = '/content/drive/MyDrive/classifier_model/finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3).cuda()


Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
import pandas as pd


In [6]:
df=pd.read_csv("dataSpecial.csv", encoding="latin", header=[0])
df

,Description,Semantic
0,Finnish Talentum reports its operating profit ...,positive
1,"Lifetree was founded in 2000 , and its revenue...",positive
2,Nokia also noted the average selling price of ...,positive
3,Calls to the switchboard and directory service...,negative
4,"Earnings per share EPS are seen at EUR 0.56 , ...",positive
5,The growth of net sales has continued favourab...,positive
6,The company slipped to an operating loss of EU...,negative
7,The company 's profit before taxes fell to EUR...,negative
8,Unit costs for flight operations fell by 6.4 p...,negative
9,"Tiimari , the Finnish retailer , reported to h...",positive


In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
import torch
import scipy as sp

In [9]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'neutral', 'positive'], use_unk_distribution=True)

In [10]:
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

In [11]:
def f2(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return int(val)

In [12]:
desc=df['Description']

In [31]:
x=desc[5]
print(x)

The growth of net sales has continued favourably in the Middle East and Africaand in Asia Pacific .


In [ ]:
pip install gensim

In [32]:
from gensim.parsing.preprocessing import remove_stopwords

y = remove_stopwords(x)

print(y)


The growth net sales continued favourably Middle East Africaand Asia Pacific .


In [33]:
pred = explainer.class_names[f2([y])]
print(pred)
exp = explainer.explain_instance(y, f, threshold=0.95)
print('Anchor: %s' % (' *** '.join(exp.names())))
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

negative
Anchor: . *** continued *** favourably *** East *** net *** Asia *** sales *** growth *** The *** Africaand *** Pacific *** Middle
Examples where anchor applies and model predicts negative:



The UNK UNK UNK continued favourably UNK UNK UNK UNK Pacific .
The UNK UNK sales UNK UNK UNK UNK Africaand UNK Pacific .
UNK growth net sales UNK favourably UNK East Africaand Asia UNK .
The growth UNK UNK continued UNK UNK East UNK Asia UNK .
UNK growth net sales continued UNK UNK UNK Africaand UNK Pacific .
UNK UNK net sales UNK UNK UNK East UNK Asia UNK .
UNK growth net sales continued UNK Middle East UNK UNK UNK .
UNK UNK UNK sales continued UNK Middle UNK UNK Asia UNK .
UNK UNK net UNK continued UNK UNK UNK UNK Asia Pacific .
The UNK net UNK UNK UNK UNK East Africaand UNK UNK .
